In [4]:
import matplotlib.pyplot as plt    
import pandas as pd
from neo4j import GraphDatabase
import sys

import os
import requests
import json
import requests
import numpy as np
from graphdatascience import GraphDataScience
from dotenv import load_dotenv
import openai
load_dotenv(override=True)

openai.api_key = os.environ['OPENAI_API_KEY']
database = 'neo4j' 
uri, user, password = os.getenv('NEO4J_URI'), os.getenv('NEO4J_USER'), os.getenv('NEO4J_PASSWORD')
print(user, password, database)
gds = GraphDataScience(
    uri,
    auth = (user, password), database=database
)       

abhishek @Neo4j123 neo4j


In [71]:
query = """MATCH (g:Genre)-[:GENRE]->(m:Movie)<-[:RATING]-(u:User)
WITH g, COUNT(DISTINCT u) AS user_count
ORDER BY user_count DESC
LIMIT 10
MATCH (genre:Genre {name: g.name})-[:GENRE]->(movie:Movie)
WHERE NOT EXISTS {
    MATCH (movie)<-[:RATING]-(user:User {email: 'abhi123@yahoo.com'})
}
WITH genre.name AS Genre, movie.name AS TopMovies, movie.popularity AS Popularity, movie.id AS movieId
ORDER BY genre.name, Popularity DESC
WITH Genre, COLLECT({movieName: TopMovies, popularity: Popularity, movieId: movieId})[0..10] AS genreMovies
RETURN Genre, genreMovies
LIMIT 10;
"""



df = pd.concat([pd.DataFrame(x) for x in gds.run_cypher(query,params = {"email":'abhi@yahoo.com'}).iloc[:,1]])
df.drop_duplicates('movieId',inplace=True)
df = df.sample(n=50, random_state=42)  # Adjust `n` as needed, and `random_state` for reproducibility
df.sort_values('movieName',inplace=True)
print(df)

                         movieName  movieId  popularity
0            2001: A Space Odyssey       62   22.494622
2             Addams Family Values     2758   17.555909
4                          Aladdin      812   16.357419
8                        Apollo 13      568   18.761467
6                       Barbarella     8069   14.304205
2                           Batman      268   19.106730
0                     Blade Runner       78   96.272374
1                       Braveheart      197   20.755149
8                     Broken Arrow     9208   14.957626
7                   Chain Reaction    12123   15.874491
8                       Cinderella    11224   14.436858
4                     Citizen Kane       15   15.811921
7                   Color of Night     2124   14.228963
6                     First Knight     6520   15.178338
9          For Whom the Bell Tolls    27854   14.373861
5                     Forget Paris    10525   16.236745
0                     Forrest Gump       13   48

## Checking Non Repetitive movies whene a user logs in

In [64]:
query = """MATCH (g:Genre)-[:GENRE]->(m:Movie)<-[:RATING]-(u:User)
WITH g, COUNT(DISTINCT u) AS user_count
ORDER BY user_count DESC
LIMIT 10
MATCH (genre:Genre {name: g.name})-[:GENRE]->(movie:Movie)<-[:RATING]-(u:User)
WITH genre.name AS Genre, movie.name AS TopMovies, movie.popularity AS Popularity, movie.id AS movieId
ORDER BY genre.name, Popularity DES
WITH Genre, COLLECT({movieName: TopMovies, popularity: Popularity, movieId: movieId})[0..10] AS genreMovies, userMovies
RETURN Genre, genreMovies,userMovies
LIMIT 10;"""

query2 = """MATCH (movie)<-[:RATING]-(user:User {email: $email}) WITH movie.name as userMovies"""

params = {"email": 'abhi123@yahoo.com'}
df = pd.concat([pd.DataFrame(x) for x in gds.run_cypher(query,params).iloc[:,1]])
df.drop_duplicates('movieId',inplace=True)
#df = df.sample(n=50, random_state=42)  # Adjust `n` as needed, and `random_state` for reproducibility
print(df)

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Variable `g` not defined (line 6, column 27 (offset: 240))
"MATCH (genre:Genre {name: g.name})-[:GENRE]->(movie:Movie)<-[:RATING]-(u:User)"
                           ^}

In [67]:
query2 = """MATCH (movie)<-[:RATING]-(user:User {email: $email}) Return movie.name as userMovies"""
params = {"email": 'abhi123@yahoo.com'}
df2 = gds.run_cypher(query2,params)
selectedMovies = df2.iloc[:,0].to_list()

query = """MATCH (g:Genre)-[:GENRE]->(m:Movie)<-[:RATING]-(u:User)
    WITH g, COUNT(DISTINCT u) AS user_count
    ORDER BY user_count DESC
    LIMIT 10
    MATCH (genre:Genre {name: g.name})-[:GENRE]->(movie:Movie)
    WITH genre.name AS Genre, movie.name AS TopMovies, movie.popularity AS Popularity,movie.id as movieId
    ORDER BY genre.name, Popularity DESC
    WITH Genre , COLLECT({movieName:TopMovies, popularity: Popularity,movieId:movieId})[0..10] as genreMovies
    return Genre, genreMovies
    LIMIT 10;"""
df = pd.concat([pd.DataFrame(x) for x in gds.run_cypher(query).iloc[:,1]])
df.drop_duplicates('movieId',inplace=True)
df = df.sample(n=50, random_state=42)  # Adjust `n` as needed, and `random_state` for reproducibility

print(df)
print(selectedMovies)

                         movieName  movieId  popularity
2             Addams Family Values     2758   17.555909
9                          The Net     1642   13.304342
8                Striking Distance    11074   13.390917
4                          Jumanji     8844   17.015539
4                     Citizen Kane       15   15.811921
5                 Independence Day      602   17.634197
0                     Forrest Gump       13   48.307194
8                     Frankenstein     3036   14.287753
3                           Batman      268   19.106730
8               The Usual Suspects      629   16.302466
6                          Aladdin      812   16.357419
8                   Chain Reaction    12123   15.874491
0                     Pulp Fiction      680  140.950236
6                     True Romance      319   17.189328
0                     Blade Runner       78   96.272374
4                             Heat      949   17.924927
9                        Apollo 13      568   18

In [52]:
query = """MATCH (g:Genre)-[:GENRE]->(m:Movie)<-[:RATING]-(u:User)
    WITH g, COUNT(DISTINCT u) AS user_count
    ORDER BY user_count DESC
    LIMIT 10
    MATCH (genre:Genre {name: g.name})-[:GENRE]->(movie:Movie)
    WITH genre.name AS Genre, movie.name AS TopMovies, movie.popularity AS Popularity,movie.id as movieId
    ORDER BY genre.name, Popularity DESC
    WITH Genre , COLLECT({movieName:TopMovies, popularity: Popularity,movieId:movieId})[0..10] as genreMovies
    return Genre, genreMovies
    LIMIT 10;"""

df = pd.concat([pd.DataFrame(x) for x in gds.run_cypher(query).iloc[:,1]])
df.drop_duplicates('movieId',inplace=True)
df = df.sample(n=50, random_state=42)  # Adjust `n` as needed, and `random_state` for reproducibility
print(df)

                         movieName  movieId  popularity
2             Addams Family Values     2758   17.555909
9                          The Net     1642   13.304342
8                Striking Distance    11074   13.390917
4                          Jumanji     8844   17.015539
4                     Citizen Kane       15   15.811921
5                 Independence Day      602   17.634197
0                     Forrest Gump       13   48.307194
8                     Frankenstein     3036   14.287753
3                           Batman      268   19.106730
8               The Usual Suspects      629   16.302466
6                          Aladdin      812   16.357419
8                   Chain Reaction    12123   15.874491
0                     Pulp Fiction      680  140.950236
6                     True Romance      319   17.189328
0                     Blade Runner       78   96.272374
4                             Heat      949   17.924927
9                        Apollo 13      568   18

In [17]:
print(similarity)

def getImage(movieId):
    
    '''
    Get Images using the TMDB API
    '''
    
    url = f"https://api.themoviedb.org/3/movie/{movieId}/images"+"?api_key=1a23dabc3e8f21345d6e1efb8b47db48"    
    headers = {
        "accept": "application/json",
        "Authorization": "Bearer "+os.getenv('MOVIE_API_TOKEN')
    }    
    response = requests.get(url) 
    print(response)
    vals = json.loads(response.text)['posters']    
    vals = ['http://image.tmdb.org/t/p/w185/' + x['file_path'] if x['iso_639_1']=='en' else 'http://image.tmdb.org/t/p/w185/' + x['file_path'] for x in vals]
    return vals[0]


similarity['posterLink'] = similarity['movieId'].apply(lambda x : getImage(x))  
print(similarity)

          Movie1                                   Movie2  movieId  \
0    Baby Driver                                    Ted 2   214756   
1    Baby Driver                             Wonder Woman   297762   
2          Ted 2                              Baby Driver   339403   
6   Pulp Fiction               Die Hard: With a Vengeance     1572   
7   Pulp Fiction                          P.S. I Love You     6023   
8   Pulp Fiction           Frankenstein Must Be Destroyed     3075   
9   Pulp Fiction                 A Streetcar Named Desire      702   
10  Pulp Fiction                                    Ronin     8195   
11  Pulp Fiction       The World, the Flesh and the Devil     4928   
12  Pulp Fiction                     The First Wives Club     2925   
13  Pulp Fiction                                Baise-moi     3134   
14  Pulp Fiction              Wizards of the Lost Kingdom     7234   
15  Pulp Fiction  Midnight in the Garden of Good and Evil     8197   
16   Baby Driver    